In [ ]:

!pip install tensorflow


In [3]:
import cv2
import numpy as np
import pickle

def build_squares(img):
	x, y, w, h = 420, 140, 10, 10
	d = 10
	imgCrop = None
	crop = None
	for i in range(10):
		for j in range(5):
			if np.any(imgCrop == None):
				imgCrop = img[y:y+h, x:x+w]
			else:
				imgCrop = np.hstack((imgCrop, img[y:y+h, x:x+w]))
			#print(imgCrop.shape)
			cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 1)
			x+=w+d
		if np.any(crop == None):
			crop = imgCrop
		else:
			crop = np.vstack((crop, imgCrop)) 
		imgCrop = None
		x = 420
		y+=h+d
	return crop

def get_hand_hist():
	cam = cv2.VideoCapture(1)
	if cam.read()[0]==False:
		cam = cv2.VideoCapture(0)
	x, y, w, h = 300, 100, 300, 300
	flagPressedC, flagPressedS = False, False
	imgCrop = None
	while True:
		img = cam.read()[1]
		img = cv2.flip(img, 1)
		img = cv2.resize(img, (640, 480))
		hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
		
		keypress = cv2.waitKey(1)
		if keypress == ord('c'):		
			hsvCrop = cv2.cvtColor(imgCrop, cv2.COLOR_BGR2HSV)
			flagPressedC = True
			hist = cv2.calcHist([hsvCrop], [0, 1], None, [180, 256], [0, 180, 0, 256])
			cv2.normalize(hist, hist, 0, 255, cv2.NORM_MINMAX)
		elif keypress == ord('s'):
			flagPressedS = True	
			break
		if flagPressedC:	
			dst = cv2.calcBackProject([hsv], [0, 1], hist, [0, 180, 0, 256], 1)
			dst1 = dst.copy()
			disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))
			cv2.filter2D(dst,-1,disc,dst)
			blur = cv2.GaussianBlur(dst, (11,11), 0)
			blur = cv2.medianBlur(blur, 15)
			ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
			thresh = cv2.merge((thresh,thresh,thresh))
			#cv2.imshow("res", res)
			cv2.imshow("Thresh", thresh)
		if not flagPressedS:
			imgCrop = build_squares(img)
		#cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
		cv2.imshow("Set hand histogram", img)
	cam.release()
	cv2.destroyAllWindows()
	with open("hist", "wb") as f:
		pickle.dump(hist, f)


get_hand_hist()

In [2]:
import cv2
import numpy as np
import pickle, os, sqlite3, random

image_x, image_y = 50, 50

def get_hand_hist():
	with open("hist", "rb") as f:
		hist = pickle.load(f)
	return hist

def init_create_folder_database():
	# create the folder and database if not exist
	if not os.path.exists("gestures"):
		os.mkdir("gestures")
	if not os.path.exists("gesture_db.db"):
		conn = sqlite3.connect("gesture_db.db")
		create_table_cmd = "CREATE TABLE gesture ( g_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE, g_name TEXT NOT NULL )"
		conn.execute(create_table_cmd)
		conn.commit()

def create_folder(folder_name):
	if not os.path.exists(folder_name):
		os.mkdir(folder_name)

def store_in_db(g_id, g_name):
	conn = sqlite3.connect("gesture_db.db")
	cmd = "INSERT INTO gesture (g_id, g_name) VALUES (%s, \'%s\')" % (g_id, g_name)
	try:
		conn.execute(cmd)
	except sqlite3.IntegrityError:
		choice = input("g_id already exists. Want to change the record? (y/n): ")
		if choice.lower() == 'y':
			cmd = "UPDATE gesture SET g_name = \'%s\' WHERE g_id = %s" % (g_name, g_id)
			conn.execute(cmd)
		else:
			print("Doing nothing...")
			return
	conn.commit()
	
def store_images(g_id):
	total_pics = 500
	hist = get_hand_hist()
	cam = cv2.VideoCapture(0)
	if cam.read()[0]==False:
		cam = cv2.VideoCapture(0)
	x, y, w, h = 300, 100, 300, 300

	create_folder("gestures/"+str(g_id))
	pic_no = 0
	flag_start_capturing = False
	frames = 0
	
	while True:
		img = cam.read()[1]
		img = cv2.flip(img, 1)
		imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
		dst = cv2.calcBackProject([imgHSV], [0, 1], hist, [0, 180, 0, 256], 1)
		disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))
		cv2.filter2D(dst,-1,disc,dst)
		blur = cv2.GaussianBlur(dst, (11,11), 0)
		blur = cv2.medianBlur(blur, 15)
		thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
		thresh = cv2.merge((thresh,thresh,thresh))
		thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
		thresh = thresh[y:y+h, x:x+w]
		# 1. it worked -  CHANGED THIS NEXT 2 LINEs OF CODE >> 
		tmp = cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
		contours = tmp[0] if len(tmp) == 2 else tmp[1]      
		# CHANGED THIS NEXT 2 LINEs OF CODE <<          

		if len(contours) > 0:
			contour = max(contours, key = cv2.contourArea)
			if cv2.contourArea(contour) > 10000 and frames > 50:
				x1, y1, w1, h1 = cv2.boundingRect(contour)
				pic_no += 1
				save_img = thresh[y1:y1+h1, x1:x1+w1]
				if w1 > h1:
					save_img = cv2.copyMakeBorder(save_img, int((w1-h1)/2) , int((w1-h1)/2) , 0, 0, cv2.BORDER_CONSTANT, (0, 0, 0))
				elif h1 > w1:
					save_img = cv2.copyMakeBorder(save_img, 0, 0, int((h1-w1)/2) , int((h1-w1)/2) , cv2.BORDER_CONSTANT, (0, 0, 0))
				save_img = cv2.resize(save_img, (image_x, image_y))
				rand = random.randint(0, 10)
				if rand % 2 == 0:
					save_img = cv2.flip(save_img, 1)
				cv2.putText(img, "Capturing...", (30, 60), cv2.FONT_HERSHEY_TRIPLEX, 2, (127, 255, 255))
				cv2.imwrite("gestures/"+str(g_id)+"/"+str(pic_no)+".jpg", save_img)

		cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
		cv2.putText(img, str(pic_no), (30, 400), cv2.FONT_HERSHEY_TRIPLEX, 1.5, (127, 127, 255))
		cv2.imshow("Capturing gesture", img)
		cv2.imshow("thresh", thresh)
		keypress = cv2.waitKey(1)
		if keypress == ord('c'):
			if flag_start_capturing == False:
				flag_start_capturing = True
			else:
				flag_start_capturing = False
				frames = 0
		if flag_start_capturing == True:
			frames += 1
		if pic_no == total_pics:
			break
            
		# 2. test - CHANGED THIS NEXT 2 LINEs OF CODE          
	# cam.release()
	cv2.destroyAllWindows()
		# CHANGED THIS NEXT 2 LINEs OF CODE >>
            
            
            
            
init_create_folder_database()
g_id = input("Enter gesture no.: ")
g_name = input("Enter gesture name/text: ")
store_in_db(g_id, g_name)
store_images(g_id)

Enter gesture no.: 2
Enter gesture name/text: Yooo!!!


In [3]:
import cv2, os

def flip_images():
	gest_folder = "gestures"
	images_labels = []
	images = []
	labels = []
	for g_id in os.listdir(gest_folder):
		for i in range(500):
			path = gest_folder+"/"+g_id+"/"+str(i+1)+".jpg"
			new_path = gest_folder+"/"+g_id+"/"+str(i+1+500)+".jpg"
			print(path)
			img = cv2.imread(path, 0)
			img = cv2.flip(img, 1)
			cv2.imwrite(new_path, img)

flip_images()

gestures/0/1.jpg
gestures/0/2.jpg
gestures/0/3.jpg
gestures/0/4.jpg
gestures/0/5.jpg
gestures/0/6.jpg
gestures/0/7.jpg
gestures/0/8.jpg
gestures/0/9.jpg
gestures/0/10.jpg
gestures/0/11.jpg
gestures/0/12.jpg
gestures/0/13.jpg
gestures/0/14.jpg
gestures/0/15.jpg
gestures/0/16.jpg
gestures/0/17.jpg
gestures/0/18.jpg
gestures/0/19.jpg
gestures/0/20.jpg
gestures/0/21.jpg
gestures/0/22.jpg
gestures/0/23.jpg
gestures/0/24.jpg
gestures/0/25.jpg
gestures/0/26.jpg
gestures/0/27.jpg
gestures/0/28.jpg
gestures/0/29.jpg
gestures/0/30.jpg
gestures/0/31.jpg
gestures/0/32.jpg
gestures/0/33.jpg
gestures/0/34.jpg
gestures/0/35.jpg
gestures/0/36.jpg
gestures/0/37.jpg
gestures/0/38.jpg
gestures/0/39.jpg
gestures/0/40.jpg
gestures/0/41.jpg
gestures/0/42.jpg
gestures/0/43.jpg
gestures/0/44.jpg
gestures/0/45.jpg
gestures/0/46.jpg
gestures/0/47.jpg
gestures/0/48.jpg
gestures/0/49.jpg
gestures/0/50.jpg
gestures/0/51.jpg
gestures/0/52.jpg
gestures/0/53.jpg
gestures/0/54.jpg
gestures/0/55.jpg
gestures/0/56.jpg
g

gestures/1/37.jpg
gestures/1/38.jpg
gestures/1/39.jpg
gestures/1/40.jpg
gestures/1/41.jpg
gestures/1/42.jpg
gestures/1/43.jpg
gestures/1/44.jpg
gestures/1/45.jpg
gestures/1/46.jpg
gestures/1/47.jpg
gestures/1/48.jpg
gestures/1/49.jpg
gestures/1/50.jpg
gestures/1/51.jpg
gestures/1/52.jpg
gestures/1/53.jpg
gestures/1/54.jpg
gestures/1/55.jpg
gestures/1/56.jpg
gestures/1/57.jpg
gestures/1/58.jpg
gestures/1/59.jpg
gestures/1/60.jpg
gestures/1/61.jpg
gestures/1/62.jpg
gestures/1/63.jpg
gestures/1/64.jpg
gestures/1/65.jpg
gestures/1/66.jpg
gestures/1/67.jpg
gestures/1/68.jpg
gestures/1/69.jpg
gestures/1/70.jpg
gestures/1/71.jpg
gestures/1/72.jpg
gestures/1/73.jpg
gestures/1/74.jpg
gestures/1/75.jpg
gestures/1/76.jpg
gestures/1/77.jpg
gestures/1/78.jpg
gestures/1/79.jpg
gestures/1/80.jpg
gestures/1/81.jpg
gestures/1/82.jpg
gestures/1/83.jpg
gestures/1/84.jpg
gestures/1/85.jpg
gestures/1/86.jpg
gestures/1/87.jpg
gestures/1/88.jpg
gestures/1/89.jpg
gestures/1/90.jpg
gestures/1/91.jpg
gestures/1

gestures/2/57.jpg
gestures/2/58.jpg
gestures/2/59.jpg
gestures/2/60.jpg
gestures/2/61.jpg
gestures/2/62.jpg
gestures/2/63.jpg
gestures/2/64.jpg
gestures/2/65.jpg
gestures/2/66.jpg
gestures/2/67.jpg
gestures/2/68.jpg
gestures/2/69.jpg
gestures/2/70.jpg
gestures/2/71.jpg
gestures/2/72.jpg
gestures/2/73.jpg
gestures/2/74.jpg
gestures/2/75.jpg
gestures/2/76.jpg
gestures/2/77.jpg
gestures/2/78.jpg
gestures/2/79.jpg
gestures/2/80.jpg
gestures/2/81.jpg
gestures/2/82.jpg
gestures/2/83.jpg
gestures/2/84.jpg
gestures/2/85.jpg
gestures/2/86.jpg
gestures/2/87.jpg
gestures/2/88.jpg
gestures/2/89.jpg
gestures/2/90.jpg
gestures/2/91.jpg
gestures/2/92.jpg
gestures/2/93.jpg
gestures/2/94.jpg
gestures/2/95.jpg
gestures/2/96.jpg
gestures/2/97.jpg
gestures/2/98.jpg
gestures/2/99.jpg
gestures/2/100.jpg
gestures/2/101.jpg
gestures/2/102.jpg
gestures/2/103.jpg
gestures/2/104.jpg
gestures/2/105.jpg
gestures/2/106.jpg
gestures/2/107.jpg
gestures/2/108.jpg
gestures/2/109.jpg
gestures/2/110.jpg
gestures/2/111.jp

In [4]:
import cv2
from glob import glob
import numpy as np
import random
from sklearn.utils import shuffle
import pickle
import os

def pickle_images_labels():
	images_labels = []
	images = glob("gestures/*/*.jpg")
	images.sort()
	for image in images:
		print(image)
		label = image[image.find(os.sep)+1: image.rfind(os.sep)]
		img = cv2.imread(image, 0)
		images_labels.append((np.array(img, dtype=np.uint8), int(label)))
	return images_labels

images_labels = pickle_images_labels()
images_labels = shuffle(shuffle(shuffle(shuffle(images_labels))))
images, labels = zip(*images_labels)
print("Length of images_labels", len(images_labels))

train_images = images[:int(5/6*len(images))]
print("Length of train_images", len(train_images))
with open("train_images", "wb") as f:
	pickle.dump(train_images, f)
del train_images

train_labels = labels[:int(5/6*len(labels))]
print("Length of train_labels", len(train_labels))
with open("train_labels", "wb") as f:
	pickle.dump(train_labels, f)
del train_labels

test_images = images[int(5/6*len(images)):int(11/12*len(images))]
print("Length of test_images", len(test_images))
with open("test_images", "wb") as f:
	pickle.dump(test_images, f)
del test_images

test_labels = labels[int(5/6*len(labels)):int(11/12*len(images))]
print("Length of test_labels", len(test_labels))
with open("test_labels", "wb") as f:
	pickle.dump(test_labels, f)
del test_labels

val_images = images[int(11/12*len(images)):]
print("Length of test_images", len(val_images))
with open("val_images", "wb") as f:
	pickle.dump(val_images, f)
del val_images

val_labels = labels[int(11/12*len(labels)):]
print("Length of val_labels", len(val_labels))
with open("val_labels", "wb") as f:
	pickle.dump(val_labels, f)
del val_labels

gestures\0\1.jpg
gestures\0\10.jpg
gestures\0\100.jpg
gestures\0\1000.jpg
gestures\0\101.jpg
gestures\0\102.jpg
gestures\0\103.jpg
gestures\0\104.jpg
gestures\0\105.jpg
gestures\0\106.jpg
gestures\0\107.jpg
gestures\0\108.jpg
gestures\0\109.jpg
gestures\0\11.jpg
gestures\0\110.jpg
gestures\0\111.jpg
gestures\0\112.jpg
gestures\0\113.jpg
gestures\0\114.jpg
gestures\0\115.jpg
gestures\0\116.jpg
gestures\0\117.jpg
gestures\0\118.jpg
gestures\0\119.jpg
gestures\0\12.jpg
gestures\0\120.jpg
gestures\0\121.jpg
gestures\0\122.jpg
gestures\0\123.jpg
gestures\0\124.jpg
gestures\0\125.jpg
gestures\0\126.jpg
gestures\0\127.jpg
gestures\0\128.jpg
gestures\0\129.jpg
gestures\0\13.jpg
gestures\0\130.jpg
gestures\0\131.jpg
gestures\0\132.jpg
gestures\0\133.jpg
gestures\0\134.jpg
gestures\0\135.jpg
gestures\0\136.jpg
gestures\0\137.jpg
gestures\0\138.jpg
gestures\0\139.jpg
gestures\0\14.jpg
gestures\0\140.jpg
gestures\0\141.jpg
gestures\0\142.jpg
gestures\0\143.jpg
gestures\0\144.jpg
gestures\0\145.jpg

gestures\0\705.jpg
gestures\0\706.jpg
gestures\0\707.jpg
gestures\0\708.jpg
gestures\0\709.jpg
gestures\0\71.jpg
gestures\0\710.jpg
gestures\0\711.jpg
gestures\0\712.jpg
gestures\0\713.jpg
gestures\0\714.jpg
gestures\0\715.jpg
gestures\0\716.jpg
gestures\0\717.jpg
gestures\0\718.jpg
gestures\0\719.jpg
gestures\0\72.jpg
gestures\0\720.jpg
gestures\0\721.jpg
gestures\0\722.jpg
gestures\0\723.jpg
gestures\0\724.jpg
gestures\0\725.jpg
gestures\0\726.jpg
gestures\0\727.jpg
gestures\0\728.jpg
gestures\0\729.jpg
gestures\0\73.jpg
gestures\0\730.jpg
gestures\0\731.jpg
gestures\0\732.jpg
gestures\0\733.jpg
gestures\0\734.jpg
gestures\0\735.jpg
gestures\0\736.jpg
gestures\0\737.jpg
gestures\0\738.jpg
gestures\0\739.jpg
gestures\0\74.jpg
gestures\0\740.jpg
gestures\0\741.jpg
gestures\0\742.jpg
gestures\0\743.jpg
gestures\0\744.jpg
gestures\0\745.jpg
gestures\0\746.jpg
gestures\0\747.jpg
gestures\0\748.jpg
gestures\0\749.jpg
gestures\0\75.jpg
gestures\0\750.jpg
gestures\0\751.jpg
gestures\0\752.jp

gestures\1\312.jpg
gestures\1\313.jpg
gestures\1\314.jpg
gestures\1\315.jpg
gestures\1\316.jpg
gestures\1\317.jpg
gestures\1\318.jpg
gestures\1\319.jpg
gestures\1\32.jpg
gestures\1\320.jpg
gestures\1\321.jpg
gestures\1\322.jpg
gestures\1\323.jpg
gestures\1\324.jpg
gestures\1\325.jpg
gestures\1\326.jpg
gestures\1\327.jpg
gestures\1\328.jpg
gestures\1\329.jpg
gestures\1\33.jpg
gestures\1\330.jpg
gestures\1\331.jpg
gestures\1\332.jpg
gestures\1\333.jpg
gestures\1\334.jpg
gestures\1\335.jpg
gestures\1\336.jpg
gestures\1\337.jpg
gestures\1\338.jpg
gestures\1\339.jpg
gestures\1\34.jpg
gestures\1\340.jpg
gestures\1\341.jpg
gestures\1\342.jpg
gestures\1\343.jpg
gestures\1\344.jpg
gestures\1\345.jpg
gestures\1\346.jpg
gestures\1\347.jpg
gestures\1\348.jpg
gestures\1\349.jpg
gestures\1\35.jpg
gestures\1\350.jpg
gestures\1\351.jpg
gestures\1\352.jpg
gestures\1\353.jpg
gestures\1\354.jpg
gestures\1\355.jpg
gestures\1\356.jpg
gestures\1\357.jpg
gestures\1\358.jpg
gestures\1\359.jpg
gestures\1\36.jp

gestures\1\925.jpg
gestures\1\926.jpg
gestures\1\927.jpg
gestures\1\928.jpg
gestures\1\929.jpg
gestures\1\93.jpg
gestures\1\930.jpg
gestures\1\931.jpg
gestures\1\932.jpg
gestures\1\933.jpg
gestures\1\934.jpg
gestures\1\935.jpg
gestures\1\936.jpg
gestures\1\937.jpg
gestures\1\938.jpg
gestures\1\939.jpg
gestures\1\94.jpg
gestures\1\940.jpg
gestures\1\941.jpg
gestures\1\942.jpg
gestures\1\943.jpg
gestures\1\944.jpg
gestures\1\945.jpg
gestures\1\946.jpg
gestures\1\947.jpg
gestures\1\948.jpg
gestures\1\949.jpg
gestures\1\95.jpg
gestures\1\950.jpg
gestures\1\951.jpg
gestures\1\952.jpg
gestures\1\953.jpg
gestures\1\954.jpg
gestures\1\955.jpg
gestures\1\956.jpg
gestures\1\957.jpg
gestures\1\958.jpg
gestures\1\959.jpg
gestures\1\96.jpg
gestures\1\960.jpg
gestures\1\961.jpg
gestures\1\962.jpg
gestures\1\963.jpg
gestures\1\964.jpg
gestures\1\965.jpg
gestures\1\966.jpg
gestures\1\967.jpg
gestures\1\968.jpg
gestures\1\969.jpg
gestures\1\97.jpg
gestures\1\970.jpg
gestures\1\971.jpg
gestures\1\972.jp

gestures\2\453.jpg
gestures\2\454.jpg
gestures\2\455.jpg
gestures\2\456.jpg
gestures\2\457.jpg
gestures\2\458.jpg
gestures\2\459.jpg
gestures\2\46.jpg
gestures\2\460.jpg
gestures\2\461.jpg
gestures\2\462.jpg
gestures\2\463.jpg
gestures\2\464.jpg
gestures\2\465.jpg
gestures\2\466.jpg
gestures\2\467.jpg
gestures\2\468.jpg
gestures\2\469.jpg
gestures\2\47.jpg
gestures\2\470.jpg
gestures\2\471.jpg
gestures\2\472.jpg
gestures\2\473.jpg
gestures\2\474.jpg
gestures\2\475.jpg
gestures\2\476.jpg
gestures\2\477.jpg
gestures\2\478.jpg
gestures\2\479.jpg
gestures\2\48.jpg
gestures\2\480.jpg
gestures\2\481.jpg
gestures\2\482.jpg
gestures\2\483.jpg
gestures\2\484.jpg
gestures\2\485.jpg
gestures\2\486.jpg
gestures\2\487.jpg
gestures\2\488.jpg
gestures\2\489.jpg
gestures\2\49.jpg
gestures\2\490.jpg
gestures\2\491.jpg
gestures\2\492.jpg
gestures\2\493.jpg
gestures\2\494.jpg
gestures\2\495.jpg
gestures\2\496.jpg
gestures\2\497.jpg
gestures\2\498.jpg
gestures\2\499.jpg
gestures\2\5.jpg
gestures\2\50.jpg


gestures\2\894.jpg
gestures\2\895.jpg
gestures\2\896.jpg
gestures\2\897.jpg
gestures\2\898.jpg
gestures\2\899.jpg
gestures\2\9.jpg
gestures\2\90.jpg
gestures\2\900.jpg
gestures\2\901.jpg
gestures\2\902.jpg
gestures\2\903.jpg
gestures\2\904.jpg
gestures\2\905.jpg
gestures\2\906.jpg
gestures\2\907.jpg
gestures\2\908.jpg
gestures\2\909.jpg
gestures\2\91.jpg
gestures\2\910.jpg
gestures\2\911.jpg
gestures\2\912.jpg
gestures\2\913.jpg
gestures\2\914.jpg
gestures\2\915.jpg
gestures\2\916.jpg
gestures\2\917.jpg
gestures\2\918.jpg
gestures\2\919.jpg
gestures\2\92.jpg
gestures\2\920.jpg
gestures\2\921.jpg
gestures\2\922.jpg
gestures\2\923.jpg
gestures\2\924.jpg
gestures\2\925.jpg
gestures\2\926.jpg
gestures\2\927.jpg
gestures\2\928.jpg
gestures\2\929.jpg
gestures\2\93.jpg
gestures\2\930.jpg
gestures\2\931.jpg
gestures\2\932.jpg
gestures\2\933.jpg
gestures\2\934.jpg
gestures\2\935.jpg
gestures\2\936.jpg
gestures\2\937.jpg
gestures\2\938.jpg
gestures\2\939.jpg
gestures\2\94.jpg
gestures\2\940.jpg


In [4]:
import cv2, os, random
import numpy as np

def get_image_size():
	img = cv2.imread('gestures/2/111.jpg', 0)
	return img.shape

gestures = os.listdir('gestures/')
gestures.sort(key = int)
begin_index = 0
end_index = 5
image_x, image_y = get_image_size()

if len(gestures)%5 != 0:
	rows = int(len(gestures)/5)+1
else:
	rows = int(len(gestures)/5)

full_img = None
for i in range(rows):
	col_img = None
	for j in range(begin_index, end_index):
		img_path = "gestures/%s/%d.jpg" % (j, random.randint(1, 500))
		img = cv2.imread(img_path, 0)
		if np.any(img == None):
			img = np.zeros((image_y, image_x), dtype = np.uint8)
		if np.any(col_img == None):
			col_img = img
		else:
			col_img = np.hstack((col_img, img))

	begin_index += 5
	end_index += 5
	if np.any(full_img == None):
		full_img = col_img
	else:
		full_img = np.vstack((full_img, col_img))


cv2.imshow("gestures", full_img)
cv2.imwrite('full_img.jpg', full_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pydot

In [11]:
import numpy as np
import pickle
import cv2, os
from glob import glob
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import backend as K
K.set_image_dim_ordering('tf')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

def get_image_size():
	img = cv2.imread('gestures/2/111.jpg', 0)
	return img.shape

def get_num_of_classes():
	return len(glob('gestures/*'))

image_x, image_y = get_image_size()

def cnn_model():
	num_of_classes = get_num_of_classes()
	model = Sequential()
	model.add(Conv2D(16, (2,2), input_shape=(image_x, image_y, 1), activation='relu'))
	model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
	model.add(Conv2D(32, (3,3), activation='relu'))
	model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same'))
	model.add(Conv2D(64, (5,5), activation='relu'))
	model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
	model.add(Flatten())
	model.add(Dense(128, activation='relu'))
	model.add(Dropout(0.2))
	model.add(Dense(num_of_classes, activation='softmax'))
	sgd = optimizers.SGD(lr=1e-2)
	model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
	filepath="cnn_model_keras2.h5"
	checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
	callbacks_list = [checkpoint1]
	from keras.utils import plot_model
	plot_model(model, to_file='model.png', show_shapes=True)
	return model, callbacks_list

def train():
	with open("train_images", "rb") as f:
		train_images = np.array(pickle.load(f))
	with open("train_labels", "rb") as f:
		train_labels = np.array(pickle.load(f), dtype=np.int32)

	with open("val_images", "rb") as f:
		val_images = np.array(pickle.load(f))
	with open("val_labels", "rb") as f:
		val_labels = np.array(pickle.load(f), dtype=np.int32)

	train_images = np.reshape(train_images, (train_images.shape[0], image_x, image_y, 1))
	val_images = np.reshape(val_images, (val_images.shape[0], image_x, image_y, 1))
	train_labels = np_utils.to_categorical(train_labels)
	val_labels = np_utils.to_categorical(val_labels)

	print(val_labels.shape)

	model, callbacks_list = cnn_model()
	model.summary()
	model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=100, batch_size=500, callbacks=callbacks_list)
	scores = model.evaluate(val_images, val_labels, verbose=0)
	print("CNN Error: %.2f%%" % (100-scores[1]*100))
	#model.save('cnn_model_keras2.h5')

train()
K.clear_session();




Using TensorFlow backend.


(250, 3)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 49, 49, 16)        80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          51264     
_________________________________________________________________
max_pooling2

2500/2500 [==============================] - 2s 614us/step - loss: 7.0661 - acc: 0.5592 - val_loss: 7.3525 - val_acc: 0.5413

Epoch 00073: val_acc did not improve from 0.56267
Epoch 74/100
2500/2500 [==============================] - 2s 610us/step - loss: 7.0661 - acc: 0.5592 - val_loss: 7.3525 - val_acc: 0.5413

Epoch 00074: val_acc did not improve from 0.56267
Epoch 75/100
2500/2500 [==============================] - 2s 647us/step - loss: 7.0661 - acc: 0.5592 - val_loss: 7.3525 - val_acc: 0.5413

Epoch 00075: val_acc did not improve from 0.56267
Epoch 76/100
2500/2500 [==============================] - 2s 630us/step - loss: 7.0661 - acc: 0.5592 - val_loss: 7.3525 - val_acc: 0.5413

Epoch 00076: val_acc did not improve from 0.56267
Epoch 77/100
2500/2500 [==============================] - 2s 602us/step - loss: 7.0661 - acc: 0.5592 - val_loss: 7.3525 - val_acc: 0.5413

Epoch 00077: val_acc did not improve from 0.56267
Epoch 78/100
2500/2500 [==============================] - 2s 608us/s

In [12]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import pickle
import numpy as np
import time
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot
    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix
    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']
    title:        the text to display at the top of the matrix
    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues
    normalize:    If False, plot the raw numbers
                  If True, plot the proportions
    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph
    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(20, 20))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.savefig('confusion_matrix.png')


image_x, image_y = 50, 50
with open("test_images", "rb") as f:
	test_images = np.array(pickle.load(f))
with open("test_labels", "rb") as f:
	test_labels = np.array(pickle.load(f), dtype=np.int32)
test_images = np.reshape(test_images, (test_images.shape[0], image_x, image_y, 1))


model = load_model('cnn_model_keras2.h5')
pred_labels = []

start_time = time.time()
pred_probabs = model.predict(test_images)
end_time = time.time()
pred_time = end_time-start_time
avg_pred_time = pred_time/test_images.shape[0]
print("Time taken to predict %d test images is %ds" %(test_images.shape[0], pred_time))
print('Average prediction time: %fs' % (avg_pred_time))

for pred_probab in pred_probabs:
	pred_labels.append(list(pred_probab).index(max(pred_probab)))

cm = confusion_matrix(test_labels, np.array(pred_labels))
classification_report = classification_report(test_labels, np.array(pred_labels))
print('\n\nClassification Report') 
print('---------------------------')
print(classification_report)
plot_confusion_matrix(cm, range(44), normalize=False)

Time taken to predict 250 test images is 0s
Average prediction time: 0.000447s


Classification Report
---------------------------
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        87
           1       0.00      0.00      0.00        75
           2       0.35      1.00      0.52        88

   micro avg       0.35      0.35      0.35       250
   macro avg       0.12      0.33      0.17       250
weighted avg       0.12      0.35      0.18       250



C:\Users\dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
cv2.__version__

In [ ]:
import cv2
import numpy as np
import pickle

def build_squares(img):
	x, y, w, h = 420, 140, 10, 10
	d = 10
	imgCrop = None
	crop = None
	for i in range(10):
		for j in range(5):
			if np.any(imgCrop == None):
				imgCrop = img[y:y+h, x:x+w]
			else:
				imgCrop = np.hstack((imgCrop, img[y:y+h, x:x+w]))
			#print(imgCrop.shape)
			cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 1)
			x+=w+d
		if np.any(crop == None):
			crop = imgCrop
		else:
			crop = np.vstack((crop, imgCrop)) 
		imgCrop = None
		x = 420
		y+=h+d
	return crop

def get_hand_hist():
	cam = cv2.VideoCapture(1)
	if cam.read()[0]==False:
		cam = cv2.VideoCapture(0)
	x, y, w, h = 300, 100, 300, 300
	flagPressedC, flagPressedS = False, False
	imgCrop = None
	while True:
		img = cam.read()[1]
		img = cv2.flip(img, 1)
		img = cv2.resize(img, (640, 480))
		hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
		
		keypress = cv2.waitKey(1)
		if keypress == ord('c'):		
			hsvCrop = cv2.cvtColor(imgCrop, cv2.COLOR_BGR2HSV)
			flagPressedC = True
			hist = cv2.calcHist([hsvCrop], [0, 1], None, [180, 256], [0, 180, 0, 256])
			cv2.normalize(hist, hist, 0, 255, cv2.NORM_MINMAX)
		elif keypress == ord('s'):
			flagPressedS = True	
			break
		if flagPressedC:	
			dst = cv2.calcBackProject([hsv], [0, 1], hist, [0, 180, 0, 256], 1)
			dst1 = dst.copy()
			disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))
			cv2.filter2D(dst,-1,disc,dst)
			blur = cv2.GaussianBlur(dst, (11,11), 0)
			blur = cv2.medianBlur(blur, 15)
			ret,thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
			thresh = cv2.merge((thresh,thresh,thresh))
			#cv2.imshow("res", res)
			cv2.imshow("Thresh", thresh)
		if not flagPressedS:
			imgCrop = build_squares(img)
		#cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
		cv2.imshow("Set hand histogram", img)
	cam.release()
	cv2.destroyAllWindows()
	with open("hist", "wb") as f:
		pickle.dump(hist, f)


get_hand_hist()

In [ ]:
!pip install pyttsx3

In [5]:
import cv2, pickle
import numpy as np
import tensorflow as tf
import os
import sqlite3, pyttsx3
from keras.models import load_model
from threading import Thread

engine = pyttsx3.init()
engine.setProperty('rate', 150)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
model = load_model('cnn_model_keras2.h5')

def get_image_size():
	img = cv2.imread('gestures/1/1000.jpg', 0)
	return img.shape

image_x, image_y = get_image_size()


def keras_process_image(img):
	img = cv2.resize(img, (image_x, image_y))
	img = np.array(img, dtype=np.float32)
	img = np.reshape(img, (1, image_x, image_y, 1))
	return img

def keras_predict(model, image):
	processed = keras_process_image(image)
	pred_probab = model.predict(processed)[0]
	pred_class = list(pred_probab).index(max(pred_probab))
	return max(pred_probab), pred_class

def get_pred_text_from_db(pred_class):
	conn = sqlite3.connect("gesture_db.db")
	cmd = "SELECT g_name FROM gesture WHERE g_id="+str(pred_class)
	cursor = conn.execute(cmd)
	for row in cursor:
		return row[0]

def split_sentence(text, num_of_words):
	'''
	Splits a text into group of num_of_words
	'''
	list_words = text.split(" ")
	length = len(list_words)
	splitted_sentence = []
	b_index = 0
	e_index = num_of_words
	while length > 0:
		part = ""
		for word in list_words[b_index:e_index]:
			part = part + " " + word
		splitted_sentence.append(part)
		b_index += num_of_words
		e_index += num_of_words
		length -= num_of_words
	return splitted_sentence

def put_splitted_text_in_blackboard(blackboard, splitted_text):
	y = 200
	for text in splitted_text:
		cv2.putText(blackboard, text, (4, y), cv2.FONT_HERSHEY_TRIPLEX, 2, (255, 255, 255))
		y += 50

def get_hand_hist():
	with open("hist", "rb") as f:
		hist = pickle.load(f)
	return hist

def recognize():
	global prediction
	cam = cv2.VideoCapture(1)
	if cam.read()[0] == False:
		cam = cv2.VideoCapture(0)
	hist = get_hand_hist()
	x, y, w, h = 300, 100, 300, 300
	while True:
		text = ""
		img = cam.read()[1]
		img = cv2.flip(img, 1)
		img = cv2.resize(img, (640, 480))
		imgCrop = img[y:y+h, x:x+w]
		imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
		dst = cv2.calcBackProject([imgHSV], [0, 1], hist, [0, 180, 0, 256], 1)
		disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10))
		cv2.filter2D(dst,-1,disc,dst)
		blur = cv2.GaussianBlur(dst, (11,11), 0)
		blur = cv2.medianBlur(blur, 15)
		thresh = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
		thresh = cv2.merge((thresh,thresh,thresh))
		thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
		thresh = thresh[y:y+h, x:x+w]
		contours = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[0]
		if len(contours) > 0:
			contour = max(contours, key = cv2.contourArea)
			#print(cv2.contourArea(contour))
			if cv2.contourArea(contour) > 10000:
				x1, y1, w1, h1 = cv2.boundingRect(contour)
				save_img = thresh[y1:y1+h1, x1:x1+w1]
				
				if w1 > h1:
					save_img = cv2.copyMakeBorder(save_img, int((w1-h1)/2) , int((w1-h1)/2) , 0, 0, cv2.BORDER_CONSTANT, (0, 0, 0))
				elif h1 > w1:
					save_img = cv2.copyMakeBorder(save_img, 0, 0, int((h1-w1)/2) , int((h1-w1)/2) , cv2.BORDER_CONSTANT, (0, 0, 0))
				
				pred_probab, pred_class = keras_predict(model, save_img)
				
				if pred_probab*100 > 80:
					text = get_pred_text_from_db(pred_class)
					print(text)
		blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
		splitted_text = split_sentence(text, 2)
		put_splitted_text_in_blackboard(blackboard, splitted_text)
		#cv2.putText(blackboard, text, (30, 200), cv2.FONT_HERSHEY_TRIPLEX, 1.3, (255, 255, 255))
		cv2.rectangle(img, (x,y), (x+w, y+h), (0,255,0), 2)
		res = np.hstack((img, blackboard))
		cv2.imshow("Recognizing gesture", res)
		cv2.imshow("thresh", thresh)
		if cv2.waitKey(1) == ord('q'):
			break

keras_predict(model, np.zeros((50, 50), dtype=np.uint8))		
recognize()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!!!
Yooo!